##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Visualización de los datos de texto en TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/text_summaries"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Utilizando la **API para resumir texto de TensorFlow**, usted puede registrar fácilmente texto arbitrario y visualizarlo en TensorBoard. Esto puede ser extremadamente útil para tomar muestras y examinar sus datos de entrada, o para registrar metadatos operativos o texto generado. También puede registrar información de diagnóstico como texto, el cual puede ser útil durante el proceso de desarrollo de su modelo.

En este tutorial, probará algunos casos de uso básicos de la API para resumir texto.

## Preparación

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
import tensorflow as tf

from datetime import datetime
import json
from packaging import version
import tempfile

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.5.0-dev20210219


## Registro de un solo fragmento de texto

Con el fin de entender cómo funciona la API para resumir texto, simplemente debe registrar un fragmento de texto y ver cómo se presenta en TensorBoard.


In [4]:
my_text = "Hello world! 😃"

In [5]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/text_basics/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  tf.summary.text("first_text", my_text, step=0)

Ahora, utilice TensorBoard para examinar el texto. Espere unos segundos para que la IU se inicie.

In [6]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/text_simple.png?raw=1"/> -->

## Organizando múltiples flujos de texto

Si tiene varios flujos de texto, puede mantenerlos en espacios de nombres separados para ayudar a organizarlos, al igual que los escalares u otros datos.

Tenga en cuenta que si registra texto con muchos pasos, TensorBoard submuestreará los pasos para mostrarlos de forma que la presentación sea manejable.  Puede controlar la frecuencia de muestreo utilizando el indicador `--samples_per_plugin`.

In [7]:
# Sets up a second directory to not overwrite the first one.
logdir = "logs/multiple_texts/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  with tf.name_scope("name_scope_1"):
    for step in range(20):
      tf.summary.text("a_stream_of_text", f"Hello from step {step}", step=step)
      tf.summary.text("another_stream_of_text", f"This can be kept separate {step}", step=step)
  with tf.name_scope("name_scope_2"):
    tf.summary.text("just_from_step_0", "This is an important announcement from step 0", step=0)
    

In [14]:
%tensorboard --logdir logs/multiple_texts --samples_per_plugin 'text=5'

## Interpretación del Markdown

TensorBoard interpreta los resúmenes de texto como Markdown, ya que un formato enriquecido puede facilitar la lectura y comprensión de los datos registrados, como se muestra a continuación. (Si no desea la interpretación Markdown, consulte [este tema](https://github.com/tensorflow/tensorboard/issues/830) para encontrar soluciones que supriman la interpretación).

In [9]:
# Sets up a third timestamped log directory under "logs"
logdir = "logs/markdown/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

some_obj_worth_noting = {
  "tfds_training_data": {
      "name": "mnist",
      "split": "train",
      "shuffle_files": "True",
  },
  "keras_optimizer": {
      "name": "Adagrad",
      "learning_rate": "0.001",
      "epsilon": 1e-07,
  },
  "hardware": "Cloud TPU",
}


# TODO: Update this example when TensorBoard is released with
# https://github.com/tensorflow/tensorboard/pull/4585
# which supports fenced codeblocks in Markdown.
def pretty_json(hp):
  json_hp = json.dumps(hp, indent=2)
  return "".join("\t" + line for line in json_hp.splitlines(True))

markdown_text = """
### Markdown Text

TensorBoard supports basic markdown syntax, including:

    preformatted code

**bold text**

| and | tables |
| ---- | ---------- |
| among | others |
"""

with file_writer.as_default():
  tf.summary.text("run_params", pretty_json(some_obj_worth_noting), step=0)
  tf.summary.text("markdown_jubiliee", markdown_text, step=0)
    

In [10]:
%tensorboard --logdir logs/markdown